In [1]:
import pandas as pd 
import scsims
from sklearn.preprocessing import LabelEncoder
import sys
import anndata as an
import scanpy as sp
import h5py

sys.path.append('../src')
sys.path.append('../tests')
from models.lib.lightning_train import DataModule, generate_trainer
from models.lib.neural import GeneClassifier
from models.lib.data import *
from models.lib.neural import *
from pytorch_lightning.loggers import WandbLogger
from torchmetrics.functional import *
import pandas as pd 

Let's do some simple preprocessing for training, and then test our library

In [2]:
# le = LabelEncoder()

# labels = pd.read_csv('../data/retina/retina_labels.tsv', sep='\t')
# labels = labels[labels['CellType'].isin(['retinal rod cell type B', 'retinal rod cell type A'])]
# labels = labels[labels['CellType'] != 'unannotated']
# labels['class_label'] = le.fit_transform(labels['CellType'])

# labels.index.name = 'cell'
# # labels = labels.iloc[0:5000, :]
# labels.to_csv('../data/retina/retina_labels_numeric.csv')

# # label_df = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# # # label_df

# # labels.to_csv('../data/retina/retina_labels_numeric.csv')

# labels = pd.read_csv('../data/retina/retina_labels_numeric.csv')
# labels.loc[:, 'class_label']

In [3]:
labels = pd.read_csv('../data/retina/raw_labels.tsv', sep='\t')
corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv')

In [4]:
# from sklearn.model_selection import train_test_split
# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

In [5]:
# corrected = pd.read_csv('../data/retina/retina_labels_numeric.csv', index_col='cell')
# current_labels = corrected['CellType']

# trainsplit, valsplit = train_test_split(current_labels, stratify=current_labels, random_state=42)
# trainsplit

In [6]:
module = DataModule(
    datafiles=['../data/retina/retina_T.h5ad'],
    labelfiles=['../data/retina/retina_labels_numeric.csv'],
    class_label='class_label',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    skip=3,
    shuffle=True,
    drop_last=True,
    normalize=True,
)

In [7]:
tabnetmodel = TabNetLightning(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    optim_params={
        'optimizer': torch.optim.Adam,
        'lr': 0.2,
        'weight_decay': 0,
    },
    scheduler_params={
        'scheduler': torch.optim.lr_scheduler.StepLR,
        'step_size': 1e-5,
    },
    metrics={
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
    },
)

wandb_logger = WandbLogger(
    project=f"tabnet-classifer-sweep",
    name='local-retina-model'
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=100,
)

Initializing network
Initializing explain matrix


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [8]:
trainer.fit(tabnetmodel, datamodule=module)

Creating train/val/test DataLoaders...


TypeError: __init__() got an unexpected keyword argument 'batch_size'